# Detecting S1000D DMC Generes

In [ ]:
!pip install datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 01 Data Collection

In [ ]:
import os
import re
# Define paths for the three folders
bike = '/content/drive/MyDrive/Bike Data Set for Release number 6 R2'
s1kd_tools ='/content/drive/MyDrive/s1kdtools'
fossig = '/content/drive/MyDrive/fossig'
info_codes_csv ='/content/drive/MyDrive/S1000D Issue 6 3498-3547.csv'

# 02 Data Pre-process

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# 建立 DataFrame 資料
data_primary_codes = {
    "label": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, "C"],  # Mixed types
    "text": [
        "Function, data for plans and description",
        "Operation",
        "Servicing",
        "Examinations, tests and checks",
        "Fault reports and isolation procedures",
        "Disconnect, remove and disassemble procedures",
        "Repairs and locally make procedures and data",
        "Assemble, install and connect procedures",
        "Package, handling, storage and transportation",
        "Miscellaneous",
        "Computer systems, software and data"
    ]
}

# Create DataFrame
df_primary_codes = pd.DataFrame(data_primary_codes)
df_primary_codes['label'] = df_primary_codes['label'].astype(str)  # Ensure uniform type
# Encode labels
label_encoder = LabelEncoder()
df_primary_codes['label'] = label_encoder.fit_transform(df_primary_codes['label'])

df_primary_codes = df_primary_codes.sample(frac=5, replace=True, random_state=42)
# Check result
df_primary_codes
print("Label mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

Label mapping: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'C': 10}


In [ ]:
import pandas as pd

df = pd.read_csv(info_codes_csv,skiprows=39) #skip first 39 rows
df.columns = ['Primary Info Code', 'Full Info Code','Description','Description(continued)'] #Column names
df = (
    df.fillna('')  # Replace NaN with empty strings
    .drop(columns=['Primary Info Code','Description(continued)'])  # Drop the 'Description(continued)' column
    .loc[
        (df['Description'].notna()) &  # Keep rows where Description is not NaN
        (df['Description'] != '-') &  # Exclude rows where Description is '-'
        (df['Description'] != '') &  # Exclude rows where Description is an empty string
        ~df['Description'].str.contains('Not available for projects', case=False, na=False)&
        ~df['Full Info Code'].str.contains('Secondary', na=False) &  # Exclude rows with specific Secondary Code patterns
        ~((df['Primary Info Code'].isna() | (df['Primary Info Code'] == '')) &  # Exclude rows where both Primary and Secondary Code are empty
          (df['Full Info Code'].isna() | (df['Full Info Code'] == '')))
    ]
)
df['Full Info Code'] = df['Full Info Code'].astype(str).str.zfill(3)
df = df[df['Full Info Code'] != '000']
df

,Full Info Code,Description
0,001,Title page
6,002,List of pages or data modules
17,003,Change record or highlights
24,004,Access illustration
27,005,List of abbreviations
...,...,...
2117,C90,Miscellaneous
2118,C91,Quality assurance
2121,C92,Vendor information
2125,C95,Naming conventions


In [ ]:
# Extract the first digit from the second column (Secondary Code)
df['label'] = df.iloc[:, 0].astype(str).str[0]  # First digit

# Extract the description from the third column
df['text'] = df.iloc[:, 1] # Third column (Description)

# Create a new DataFrame containing only the extracted information
df_extracted = df[['label', 'text']]
df_extracted['label'] = df_extracted['label'].astype(str)  # Ensure uniform type
# Encode labels
label_encoder = LabelEncoder()
df_extracted['label'] = label_encoder.fit_transform(df_extracted['label'])
df_extracted

<ipython-input-97-81f70ed9a736>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['label'] = df_extracted['label'].astype(str)  # Ensure uniform type
<ipython-input-97-81f70ed9a736>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['label'] = label_encoder.fit_transform(df_extracted['label'])


,label,text
0,0,Title page
6,0,List of pages or data modules
17,0,Change record or highlights
24,0,Access illustration
27,0,List of abbreviations
...,...,...
2117,10,Miscellaneous
2118,10,Quality assurance
2121,10,Vendor information
2125,10,Naming conventions


In [ ]:
# Extract the first digit from the second column (Secondary Code)
df['label'] = df.iloc[:, 0].astype(str).str[0]  # First digit

# Extract the description from the third column
df['text'] = df.iloc[:, 1] # Third column (Description)

# Create a new DataFrame containing only the extracted information
df_extracted = df[['label', 'text']]
df_extracted['label'] = df_extracted['label'].astype(str)  # Ensure uniform type
# Encode labels
label_encoder = LabelEncoder()
df_extracted['label'] = label_encoder.fit_transform(df_extracted['label'])
df_extracted

<ipython-input-98-81f70ed9a736>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['label'] = df_extracted['label'].astype(str)  # Ensure uniform type
<ipython-input-98-81f70ed9a736>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['label'] = label_encoder.fit_transform(df_extracted['label'])


,label,text
0,0,Title page
6,0,List of pages or data modules
17,0,Change record or highlights
24,0,Access illustration
27,0,List of abbreviations
...,...,...
2117,10,Miscellaneous
2118,10,Quality assurance
2121,10,Vendor information
2125,10,Naming conventions


In [ ]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Regular expression pattern to extract the dmCode attributes (infoCode)
dm_code_pattern = r'<dmCode\s+.*?\s+infoCode="(\d+)'

# List to store the extracted data
data = []

folders = [
    bike,
    s1kd_tools,
    fossig
]

# Iterate over each folder
for folder in folders:
    # List all files in the folder
    files = os.listdir(folder)

    # Filter out XML files
    xml_files = [file for file in files if file.endswith('.XML')]

    # Iterate through each XML file in the current folder
    for xml_file in xml_files:
        file_path = os.path.join(folder, xml_file)

        # Open and parse the XML file
        with open(file_path, 'r', encoding='utf-8') as file:
            xml_content = file.read()

        # Parse the XML using BeautifulSoup
        soup = BeautifulSoup(xml_content, 'xml')

        # Extract infoCode (first digit only)
        dm_code_tag = soup.find('dmCode')
        if dm_code_tag and dm_code_tag.get('infoCode'):
            info_code = dm_code_tag['infoCode'][0]  # Extract the first digit
        else:
            continue  # Skip if no valid infoCode is found

        # Extract content from <infoName> tag
        content_tag = soup.find('infoName')
        if content_tag:
            # Remove any <identAndStatusSection> content if present
            ident_and_status_section = content_tag.find('identAndStatusSection')
            if ident_and_status_section:
                ident_and_status_section.decompose()  # Remove it

            # Extract plain text from the remaining content
            text = content_tag.get_text(separator=" ", strip=True)
        else:
            text = ""

        # Append the result to the data list
        data.append({'label': info_code, 'text': text})

# Convert the list into a DataFrame
extracted_df = pd.DataFrame(data)
extracted_df['label'] = extracted_df['label'].astype(str)  # Ensure uniform type

# Encode labels
label_encoder = LabelEncoder()
extracted_df['label'] = label_encoder.fit_transform(extracted_df['label'])

# Display the DataFrame
extracted_df

,label,text
0,0,Products cross-reference table
1,1,Normal operation procedures (crew)
2,0,Copyright
3,8,Replacement procedure
4,0,Business rules document
...,...,...
169,0,Title page
170,0,Business rules
171,0,FOSSIG business rule decisions
172,0,Title page


In [ ]:
# Append the DataFrames
dataset_pd = pd.concat([df_extracted, df_primary_codes,extracted_df], ignore_index=True)

# Shuffle the combined DataFrame
dataset_pd = dataset_pd.sample(frac=1, random_state=42).reset_index(drop=True)
dataset_pd

,label,text
0,0,Functional item numbers common information rep...
1,10,Miscellaneous parts list associated with compu...
2,8,"Package, handling, storage and transportation"
3,1,Operation
4,1,Pre-operation procedure
...,...,...
787,0,List of illustrations (normally used in front ...
788,0,Zonal maintenance/inspection tasks list
789,3,Other check
790,7,Parts list associated with installation


In [ ]:
from sklearn.model_selection import train_test_split

finetune_df, test_df = train_test_split(dataset_pd, test_size=0.2)

# Ensure all labels are strings
finetune_df['label'] = finetune_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)

# Encode labels into integers
label_encoder = LabelEncoder()
finetune_df['label'] = label_encoder.fit_transform(finetune_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])  # Use the same encoder
finetune_df, test_df = train_test_split(dataset_pd, test_size=0.2, random_state=42)


In [ ]:
finetune_df

,label,text
342,7,Close after access procedure
244,2,Remove ice
314,0,Applicability cross-reference table
727,9,Generic process
691,1,Riding a bicycle
...,...,...
71,0,Description of how it is made
106,7,Preparation before installation
270,2,Release oxygen pressure
435,0,Description


In [ ]:
test_df

,label,text
694,9,Disposal of Product
528,2,Flush
598,8,Maintenance Allocation Chart
606,6,Burnish
120,0,Fasteners
...,...,...
433,0,Configuration
467,0,Highlights
361,1,Modes of operation
79,0,List of support equipment (normally used in fr...


In [ ]:
from datasets import Dataset, DatasetDict

dataset_dict = {'finetune': Dataset.from_pandas(finetune_df),
                'test' : Dataset.from_pandas(test_df)}

dataset = DatasetDict(dataset_dict)

#  Models +  Matrics

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
import matplotlib.pyplot as plt
import nltk
import math
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Download NLTK Resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
#TF-IDF Function
def custom_tfidf(input_corpus):
    vectorizer = CountVectorizer()
    term_freq_matrix = vectorizer.fit_transform(input_corpus).toarray()
    feature_names = vectorizer.get_feature_names_out()

    # Document count
    doc_count = len(input_corpus)
    df = (term_freq_matrix > 0).sum(axis=0)

    # IDF Calculation
    idf = [math.log10(doc_count / freq) if freq > 0 else 0 for freq in df]

    # TF-IDF Matrix
    tfidf_matrix = term_freq_matrix * idf

    # Create DataFrame
    return pd.DataFrame(tfidf_matrix, columns=feature_names, index=['Training Dataset','Test Dataset'])

In [ ]:
corpus = [finetune_df['text'].str.cat(sep=' '),
          test_df['text'].str.cat(sep=' ')]

In [ ]:
custom_tfidf_df = custom_tfidf(corpus)
custom_tfidf_df

,134,abbreviations,abrasive,acceptance,access,accidental,accomplishment,act,activate,add,...,wax,weld,when,wire,wiring,with,without,worksheets,zonal,zones
Training Dataset,0.30103,0.0,0.60206,0.30103,0.0,0.30103,0.30103,0.30103,0.30103,0.30103,...,0.00000,0.30103,1.20412,0.60206,0.60206,0.0,0.30103,0.30103,0.00000,0.0
Test Dataset,0.00000,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.30103,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.30103,0.0


In [ ]:
# Visualizing TF-IDF Results
def visualize_tfidf(tfidf_df, doc_label, title):
    top_terms = tfidf_df.loc[doc_label].sort_values(ascending=False).head(10)
    plt.figure(figsize=(7, 3))
    plt.bar(top_terms.index, top_terms.values)
    plt.title(title)
    plt.ylabel("TF-IDF Score")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB


In [ ]:
# Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(corpus, labels, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=2000)
X_train = vectorizer.fit_transform(finetune_df['text'])
X_test = vectorizer.transform(test_df['text'])
y_train = finetune_df['label']
y_test = test_df['label']

# 檢查 y_train 的標籤
print("Unique labels in y_train:", np.unique(y_train))
print("Label mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

# 根據 y_train 中的實際標籤調整 sampling_strategy
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 確認平衡後的類別分佈
print("Before SMOTE:", pd.Series(y_train).value_counts())
print("After SMOTE:", pd.Series(y_train_resampled).value_counts())

# Step 5: Train classifiers
classifiers = {
    "SVM": SVC(probability=True, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "Naive Bayes": MultinomialNB()
}

trained_classifiers = {}
for name, clf in classifiers.items():
    clf.fit(X_train_resampled, y_train_resampled)
    trained_classifiers[name] = clf



# Step 6: Evaluate classifiers
metrics = {"Accuracy": [], "Precision": [], "Recall": [], "F1": []}
for name, clf in trained_classifiers.items():
    y_pred = clf.predict(X_test)
    metrics["Accuracy"].append(accuracy_score(y_test, y_pred))
    metrics["Precision"].append(precision_score(y_test, y_pred, average='weighted'))
    metrics["Recall"].append(recall_score(y_test, y_pred, average='weighted'))
    metrics["F1"].append(f1_score(y_test, y_pred, average='weighted'))

metrics_df = pd.DataFrame(metrics, index=classifiers.keys())
print(metrics_df)

from sklearn.metrics import classification_report
for name, clf in trained_classifiers.items():
    y_pred = clf.predict(X_test)
    report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
    print(f"Classification Report for {name}:\n", report)

Unique labels in y_train: [ 0  1  2  3  4  5  6  7  8  9 10]
Label mapping: {'0': 0, '1': 1, '10': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10}
Before SMOTE: label
0     177
6      73
2      70
3      65
1      51
10     39
9      36
8      35
7      33
5      28
4      26
Name: count, dtype: int64
After SMOTE: label
7     177
2     177
0     177
9     177
1     177
3     177
6     177
10    177
5     177
8     177
4     177
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sampl

                     Accuracy  Precision    Recall        F1
SVM                  0.729560   0.756430  0.729560  0.715517
Decision Tree        0.716981   0.723264  0.716981  0.697292
Random Forest        0.748428   0.763075  0.748428  0.730676
Logistic Regression  0.729560   0.756209  0.729560  0.725430
Naive Bayes          0.654088   0.698394  0.654088  0.657660
Classification Report for SVM:
               precision    recall  f1-score   support

           0       0.58      0.94      0.72        48
           1       0.83      0.67      0.74        15
          10       0.95      0.73      0.83        26
           2       0.73      0.89      0.80         9
           3       1.00      0.86      0.92         7
           4       0.88      0.78      0.82         9
           5       0.60      0.55      0.57        11
           6       1.00      0.50      0.67        10
           7       1.00      0.60      0.75         5
           8       1.00      0.58      0.74        12
       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Split the dataset
X_train = finetune_df['text']
y_train = finetune_df['label']
X_test = test_df['text']
y_test = test_df['label']

# Transform text data into numerical format
vectorizer = TfidfVectorizer(max_features=2000)  # Limit to 2000 features to reduce dimensionality
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Step 4: Train Classifiers

# Define classifiers
classifiers = {
    "SVM": SVC(probability=True, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
}

# Train classifiers
trained_classifiers = {}
for name, clf in classifiers.items():
    clf.fit(X_train_tfidf, y_train)
    trained_classifiers[name] = clf

# Step 5: Evaluate Classifiers

# Initialize a dictionary to store metrics for each classifier and each label
metrics = {
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1": []
}

for name, clf in trained_classifiers.items():
    y_pred = clf.predict(X_test_tfidf)
    metrics["Accuracy"].append(accuracy_score(y_test, y_pred))
    metrics["Precision"].append(precision_score(y_test, y_pred,average='weighted'))
    metrics["Recall"].append(recall_score(y_test, y_pred,average='weighted'))
    metrics["F1"].append(f1_score(y_test, y_pred,average='weighted'))

# Summarize Results
metrics_df = pd.DataFrame(metrics, index=classifiers.keys())
metrics_df

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install huggingface_hub
!pip install evaluate
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"],
                     padding="max_length",
                     truncation=True,
                     max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=11)

In [ ]:
# from datasets import load_metric
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

# metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(num_train_epochs=1,
                                  output_dir="output",
                                  report_to="none")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["finetune"],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,

)

In [ ]:
import numpy as np

trainer.evaluate()

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
predict_ = trainer.predict(tokenized_datasets["test"])

In [ ]:
# Softmax function
def softmax(logits):
    exp_logits = np.exp(logits)
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

# Apply softmax to the predictions
probabilities = softmax(predict_.predictions)

# Map to binary output: select class with the higher probability
binary_output = np.argmax(probabilities, axis=1)
binary_output

In [ ]:
test_df['predict'] = binary_output

In [ ]:
bert_result = test_df[test_df['label'] != test_df['predict']]
bert_result

In [ ]:
metrics = dict()
metrics["Accuracy"] = accuracy_score(test_df['label'], test_df['predict'])
metrics["Precision"] = precision_score(test_df['label'], test_df['predict'], average='weighted')
metrics["Recall"] = recall_score(test_df['label'], test_df['predict'],average='weighted')
metrics["F1"] = f1_score(test_df['label'], test_df['predict'],average='weighted')
metrics_df.loc["BERT"] = metrics
metrics_df

In [ ]:
from sklearn.metrics import classification_report  # Import the function

report = classification_report(y_test, binary_output)

# Print or return the classification report
print(report)

## Visualization